In [55]:
import pandas as pd
import chardet

#Metodo que lee el tipo de encoding para evitar errores de lectura
def smart_read_csv(file_path) -> pd.DataFrame:
    with open(file_path,'rb') as f:
        res = chardet.detect(f.read(1000))

    return pd.read_csv(file_path, encoding=res['encoding'], encoding_errors='replace')

#Leemos los documentos
try:
    df_orders = smart_read_csv('../data/orders.csv')
    df_orders_details = smart_read_csv('../data/order_details.csv')
    df_pizzas = smart_read_csv('../data/pizzas.csv')
    df_pizza_types = smart_read_csv('../data/pizza_types.csv')

    print('Files load completed succcesfully')
except Exception as e:
    print(f"Error: {e}")

Files load completed succcesfully


In [58]:
#Une la cabecera los pedidos con el detalle del pedido
df_orders_master = pd.merge(df_orders, df_orders_details, how="left", on="order_id")

#Une él codigo de pizza con el tipo de pizza que representa (sus características)
df_pizzas_master = pd.merge(df_pizzas, df_pizza_types, how= "left", on="pizza_type_id")

#Genera la sábana completa de pedidos, en la cual se añade información de las pizzas solicitadas
df_master = pd.merge(df_orders_master, df_pizzas_master, how="left", on="pizza_id")


In [ ]:
#Elimina los identificadores de pizza, su tipo e ingredientes
df_master = df_master.drop(
    columns = ['pizza_id','pizza_type_id','ingredients']
)

In [ ]:
#Concatena fecha y tiempo en una sola columna y se le convierte al tipo datetime
df_master['order_timestamp'] = df_master['date'] + " " + df_master['time']
df_master['order_timestamp'] = pd.to_datetime(df_master['order_timestamp'], errors= 'coerce')
#Se elimina las columnas fecha y hora que se encontraban separadas
df_master = df_master.drop(
    columns=['date','time']
)

In [ ]:
#Cálculo del total de linea en base a la cantidad y precio
df_master['total_line'] = df_master['quantity'] * df_master['price']

In [ ]:
#Ordena la tabla según la fecha y hora en ascendente.
df_master = df_master.sort_values('order_timestamp')

In [ ]:
#Se crea un dataframe final de ventas con las columnas ordenadas para un mayor entendimiento
columns_order = [
    'order_id', 'order_details_id', 'order_timestamp', 'name', 
    'category', 'size', 'quantity', 'price', 'total_line'
]
df_pizza_sales = df_master[columns_order]